In [15]:
import openpyxl
import pandas as pd
import datetime as dt

In [2]:
with pd.ExcelFile('excel_files/stores.xlsx', engine='openpyxl') as xlfile:
    # Read a DataFrame
    df = pd.read_excel(xlfile, sheet_name='2020')
    
    # Get the OpenPyXL workbook object
    book = xlfile.book
    
    # From here on, it's OpenPyXL code
    sheet = book['2019']
    value = sheet['B3'].value  # Read a single value

In [3]:
with pd.ExcelWriter('pandas_and_openpyxl.xlsx', engine='openpyxl') as writer:
    df = pd.DataFrame({'col1': [1, 2, 3, 4], 'col2': [5, 6, 7, 8]})
    
    # Write a DataFrame
    df.to_excel(writer, 'Sheet1', startrow=4, startcol=2)
    
    # Get the OpenPyXL workbook and sheet objects
    book = writer.book
    sheet = writer.sheets['Sheet1']
    
    # From here on, it's OpenPyXL code
    sheet['A1'].value = 'This is a Title' # Write a single cell value

/tmp/ipykernel_48175/2754380846.py:5: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, 'Sheet1', startrow=4, startcol=2)


## Adjust DataFrame index and header

In [4]:
df = pd.DataFrame({'col1': [1, -2], 'col2': [-3, 4]}, index=['row1', 'row2'])
df.index.name = 'ix'
df

,col1,col2
ix,,
row1,1,-3
row2,-2,4


### Formatting index/headers with OpenPyXL

In [5]:
from openpyxl.styles import PatternFill

In [7]:
with pd.ExcelWriter('formatting_openpyxl.xlsx', engine='openpyxl') as writer:
    # Write out the df with the default formatting to A1
    df.to_excel(writer, startrow=0, startcol=0)
    
    # Write out the df with custom index/header formatting to A6
    startrow, startcol = 0, 5
    
    # 1. Write out the data part of the DataFrame
    df.to_excel(writer, header=False, index=False, startrow=startrow + 1, startcol=startcol + 1)
    
    # Get the sheet object and create a style object
    sheet = writer.sheets['Sheet1']
    style = PatternFill(fgColor='D9D9D9', fill_type='solid')
    
    # 2. Write out the styled column headers
    for i, col in enumerate(df.columns):
        sheet.cell(row=startrow + 1, column=i + startcol + 2, value=col).fill = style
        
    # 3. Write out the styled index
    index = [df.index.name if df.index.name else None] + list(df.index)
    for i, row in enumerate(index):
        sheet.cell(row=i + startrow + 1, column=startcol + 1, value=row).fill = style

### Formatting indx/headers with XlsxWriter

In [8]:
with pd.ExcelWriter('formatting_xlsxwriter.xlsx', engine='xlsxwriter') as writer:
    # Write out the df with the default formatting to A1
    df.to_excel(writer, startrow=0, startcol=0)
    
    # Write out the df with custom index/header formatting to A6
    startrow, startcol = 0, 5
    
    # 1. Write out the data part of the DataFrame
    df.to_excel(writer, header=False, index=False, startrow=startrow + 1, startcol=startcol + 1)
    
    # Get the book and sheet object and create a style object
    book = writer.book
    sheet = writer.sheets['Sheet1']
    style = book.add_format({'bg_color': '#D9D9D9'})
    
    # 2. Write out the styled column headers
    for i, col in enumerate(df.columns):
        sheet.write(startrow, startcol + i + 1, col, style)
        
    # 3. Write out the styled index
    index = [df.index.name if df.index.name else None] + list(df.index)
    for i, row in enumerate(index):
        sheet.write(startrow + i, startcol, row, style)

## Set Data Format

### Adjusting with OpenPyXL

In [9]:
from openpyxl.styles import Alignment

In [10]:
with pd.ExcelWriter('data_format_openpyxl.xlsx', engine='openpyxl') as writer:
    # Write out the DataFrame
    df.to_excel(writer)
    
    # Get the book and sheet objects
    book = writer.book
    sheet = writer.sheets['Sheet1']
    
    # Formatting individual cells
    nrows, ncols = df.shape
    for row in range(nrows):
        for col in range(ncols):
            # +1 to account for the header/index
            # +1 since OpenPyXL is 1-based
            cell = sheet.cell(row=row + 2, column=col + 2)
            cell.number_format = '0.000'
            cell.alignment = Alignment(horizontal='center')

### Adjusting with Xlsx

In [11]:
with pd.ExcelWriter('data_format_xlsxwriter.xlsx', engine='xlsxwriter') as writer:
    # Write out the DataFrame
    df.to_excel(writer)
    
    # Get the book and sheet objects
    book = writer.book
    sheet = writer.sheets['Sheet1']
    
    # Formatting the columns (individual cells can't be formatted)
    number_format = book.add_format({'num_format': '0.000', 'align': 'center'})
    sheet.set_column(first_col=1, last_col=2, cell_format=number_format)

### Adjusting with Pandas

In [13]:
df.style.applymap(lambda x: 'number-format: 0.000;text-align: center').to_excel('styled.xlsx')

/tmp/ipykernel_48175/2213964731.py:1: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df.style.applymap(lambda x: 'number-format: 0.000;text-align: center').to_excel('styled.xlsx')


In [16]:
df = pd.DataFrame({'Date': [dt.date(2020, 1, 1)],
                   'Datetime': [dt.datetime(2020, 1, 1, 10)]})

with pd.ExcelWriter('date.xlsx', date_format='yyyy-mm-dd', datetime_format='yyyy-mm-dd hh:mm:ss') as writer:
    df.to_excel(writer)